# Trace Estimator  
https://doi.org/10.1016/0377-0427(96)00018-0 
## b) Algorithm 1 with Gauss-Radau quadrature

In [54]:
import numpy as np 

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

a = -1*np.ones(n-1)
b = 2*np.ones(n)
c = -1*np.ones(n-1)
A = tridiag(a, b, c)
A



array([[ 2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
      

In [113]:
def T_tilda(T, gamma, a):
    # Remark 3
    # tridiagonal system, could be written without np.linalg.solve maybe
    e_j = np.zeros(T.shape[0])
    e_j[-1] = 1
    delta = np.linalg.solve(T-a*np.eye(T.shape[0]), gamma*gamma*e_j)
    phi = a + delta[-1]
    temp_left = np.vstack((T, gamma*e_j))
    temp_right = np.expand_dims(np.append(gamma*e_j, phi),1)
    return np.hstack((temp_left, temp_right))

def algorithm_1(A, u, a, function, maxit=50, epsilon=1e-5):
    x_j2 = 0
    x_j1 = u/np.linalg.norm(u)    
    gamma_j = 0
    for j in range(maxit):
        alpha_j = x_j1.T@A@x_j1
        r_j = A@x_j1 - alpha_j*x_j1 - gamma_j*x_j2
        gamma_j = np.linalg.norm(r_j)
        # build T_j:
        for _ in range(2):
            if j==0:
                T_j = np.array([alpha_j])
            else:
                # print("T_j shape:", T_j.shape)
                # horizontal array [0, ..., 0, gamma_j
                temp_h = np.expand_dims(np.zeros(T_j.shape[0]),1)
                temp_h[-1] = gamma_j
                # print("temp h shape:",temp_h.shape)
                # vertical array [0, ..., 0, gamma_j, alpha_j].T
                temp_v = np.expand_dims(np.zeros(T_j.shape[0] + 1),1)
                temp_v[-1] = alpha_j
                temp_v[-2] = gamma_j
                # print("temp v shape:", temp_v.shape)
                # new T_j:
                T_j = np.hstack((np.vstack((T_j, temp_h.T)), temp_v))
            # for Gauss Radau: 
            T_tilda_j = T_tilda(T_j, gamma_j, a)
            # compute eigenvalues of T_tilda_j:
            theta_k, eig_vec = np.linalg.eig(T_tilda_j)
            w_k_square = eig_vec[0, :]*eig_vec[0,:]
            I_j = np.sum(function(theta_k)*w_k_square)
        if j>0 and np.abs(I_j - I_j1) <= epsilon*np.abs(I_j):
            break
        x_j2 = x_j1
        x_j1 = r_j/gamma_j
        I_j1 = I_j

    return u.dot(u)*I_j     
         

In [114]:
# test T_tilda function
a = 9
gamma = 3
T_tilda_j = T_tilda(A, gamma, a)
print(T_tilda_j)
print("Is a in the eigenvalues of T_tilda_j:", a in np.linalg.eigvals(T_tilda_j)) 

[[ 2.        -1.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.       ]
 [-1.         2.        -1.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.       ]
 [ 0.        -1.         2.        -1.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.       ]
 [ 0.         0.        -1.         2.        -1.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.     

In [115]:
tol = 1e-3
maxit = 10
n = 25
u = np.random.rand(n)
a=3

def f(x):
    return 1/x

L = algorithm_1(A=A, u=u, a=a, function=f, maxit=20, epsilon=tol)
print(L)

15.407060172968135


### e) Numerical experiments (first draft)

In [1]:
import time 
import numpy as np
from scipy.linalg import block_diag

In [ ]:
#cell to be rewritten, temporary assignement to A
A=np.eye(3,3) 

**The different matrices they experimented on in the paper:**

In [29]:
#example 1: the heat flow matrix

In [25]:
def heat_flow_function(nu, n):
    '''
    input:  nu: a scalar 
            n: a scalar
    output: the heat flow matrix with dimension n**2 x n**2
    '''
    vec_nu=(-nu)*np.ones(n**2-n)
    vec_a=np.ones(n-1)*(-nu)
    vec_b=np.ones(n)*(1+4*nu)
    D=tridiag(vec_a, vec_b, vec_a)
    return block_diag(*([D] * n))+np.diag(vec_nu, n)+np.diag(vec_nu,-n)

In [28]:
n=2
nu=3 #some random values for now, nu>0 implies heat_flow_matrix is positive definite which is something we want
heat_flow_matrix=heat_flow_function(2,3)
heat_flow_matrix

array([[ 9., -2.,  0., -2.,  0.,  0.,  0.,  0.,  0.],
       [-2.,  9., -2.,  0., -2.,  0.,  0.,  0.,  0.],
       [ 0., -2.,  9.,  0.,  0., -2.,  0.,  0.,  0.],
       [-2.,  0.,  0.,  9., -2.,  0., -2.,  0.,  0.],
       [ 0., -2.,  0., -2.,  9., -2.,  0., -2.,  0.],
       [ 0.,  0., -2.,  0., -2.,  9.,  0.,  0., -2.],
       [ 0.,  0.,  0., -2.,  0.,  0.,  9., -2.,  0.],
       [ 0.,  0.,  0.,  0., -2.,  0., -2.,  9., -2.],
       [ 0.,  0.,  0.,  0.,  0., -2.,  0., -2.,  9.]])

In [ ]:
#example 2: Vicsek fractal Hamiltonian (VFH) matrix)

In [ ]:
#example 3: the poisson matrix

In [ ]:
#the Wathen matrix



In [ ]:
#the Lehmer matrix

#easy to implement with a for loop but not optimal, is there a better way? Couldn't find built in ways on internet

In [33]:
#the Pei matrix
alpha=2
n=4
Pei_matrix=alpha*np.eye(n)+np.ones((4,4))

In [ ]:
#running time of algo 2

start=time.time()
E,L,U=algorithm_2(A,m,p) #for some m,p to tune 
execution_algo_2=time.time()-start 

In [ ]:
#running time using built in numpy functions

start=time.time()
Tr_A_inv=np.trace(np.linalg.inv(A))
execution_built_in=time.time()-start 

In [ ]:
#solving n linear equations

#(Armelle: in my opinion, when they say compute trace(inv(A)) using n linear equations, they mean computing 
#e_i.T inv(A) e_i for i in {1,...,n} )

start=time.time()

trace=0
for i in range(n):
    e=np.zeros(n)
    e[i]=1
    trace+=e.T@np.linalg.inv(A)@e
execution_linear_equations=time.time()-start 

In [ ]:
#using algorithm 1

n=A.shape[0]
start=time.time()
trace=0
for i in range (n):
    trace+= #output of algo 1 (I guess here also parameter values to tune)
execution_algo_1=time.time()-start 